<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/237_PredRevenue_Gap_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gap Detection Utilities for Revenue Gap Orchestrator

Below is a clean, structured breakdown of **what to learn**, **what matters**, and **how to think about gap detection in a best-in-class orchestrator**.

---

# ⭐ High-Level Goal of Gap Detection

**Revenue analysis tells us WHAT the customer is doing.
Gap detection tells us WHY we should care.**

In an orchestrator, this utility answers:

> “Is there a problem worth acting on?”

This is where your agent becomes *action-oriented* instead of just *analytical*.

---

# 🚀 What You Should Focus On and Learn

Gap detection is fundamentally about learning **decision rules**.
If you master that concept, you master the entire orchestrator design.

Here’s what you should understand deeply:

---

# 1️⃣ **How to Detect Meaningful Business Conditions**

Each function corresponds to a **business signal**:

### ✔ `detect_declining_revenue_gap()`

* Looks for % decline in recent vs. baseline spending
* Uses threshold (default −15%)
* Classifies severity (`high`, `medium`, `low`)

Learn:

* How percentage deltas are used to define business health
* How domain experts pick thresholds
* How declining trends differ from absolute drops

---

### ✔ `detect_below_baseline_gap()`

* Compares *current revenue* to the *baseline average*
* Flags gaps if below a threshold (default −20%)

Learn:

* When a decline is severe *relative to historical norms*
* How to detect underperformance even without a trend

---

### ✔ `detect_churn_risk()`

* Looks at **consecutive zero spend weeks**
* Adds soft factors:

  * Declining trend in the last 4 weeks
  * Zero-spend streak
* Returns a **probability-like risk score**

Learn:

* Behavioral signals of churn
* How multiple weak signals combine into risk
* How to tune thresholds for different industries

---

### ✔ `detect_zero_spend_gap()`

* Flags zero spend in recent period (default last 4 weeks)
* Estimates expected revenue based on baseline

Learn:

* How to detect short-term anomalies
* How "recency windows" impact gap sensitivity

---

# 2️⃣ **Rule-Based Systems as Explainable AI**

The key thing you're learning here:

> **Rule-based detection = Transparent, explainable reasoning**

Every gap includes a **rationale text**, e.g.:

> "Customer revenue declined 23.5% from baseline"

This makes the orchestrator:

* Trustworthy
* Interpretable
* Easy to debug
* Easy to tune

This is why rule-based gap detection is **industry-standard** even inside ML products.

---

# 3️⃣ **How to Orchestrate Multiple Signals into a Single Customer Profile**

`detect_all_gaps_for_customer()` shows the orchestration pattern:

```python
# 1. Declining revenue
# 2. Below baseline
# 3. Zero spend
```

Learn:

* How multiple detectors form a "diagnostic panel"
* How signals cascade: trend → baseline → zero-spend
* How to merge many signals into a coherent view

This mirrors real enterprise systems:

* Salesforce Einstein
* Adobe Customer Journey Analysis
* AWS Fraud Detector
* Shopify Intelligence

---

# 4️⃣ **How to Build for Extensibility**

This file is designed SO WELL that:

* Adding new detectors is trivial
* Adding stockout analysis is a drop-in
* An ML-powered detector can be plugged in without touching others

Example:
You can add:

### ✔ `detect_stockout_related_gap()`

using merged sales + stock data.

Or:

### ✔ `detect_anomalous_behavior_gap()` (ML anomaly detection)

This design is flexible, modular, and clean.

---

# 5️⃣ **Understanding Thresholds as Configuration**

Notice how thresholds aren’t hard-coded:

```python
threshold=gap_thresholds.get("declining_revenue_threshold", -15.0)
```

Learn:

* Why thresholds live in config, not code
* How tuning thresholds changes agent behavior
* How this makes your orchestrator adaptable across industries

**This is exactly how enterprise orchestrators are built.**

---

# 6️⃣ **Understanding Severity Classification**

Severity is determined by:

```python
if gap_percentage <= -30.0: high  
elif gap_percentage <= -15.0: medium  
else: low
```

Learn:

* How to discretize continuous values into actionable buckets
* How severity feeds directly into **scoring**
* How severity influences **ranking**
* How severity influences the **final report output**

---

# 7️⃣ **Learning the Agent Philosophy:**

**Gaps are NOT predictions — they're triggers**
This entire file teaches you:

> **"Analysis → Detect → Trigger → Prioritize → Act"**

ML predictions can feed analysis,
but gap detection decides whether to *take action*.

This separation of responsibilities is best-in-class design.

---

# 🧠 Putting It All Together

You should walk away understanding:

## ✔ Rule-based detection

How to translate business logic into rules.

## ✔ Threshold tuning

How to make the system more/less sensitive.

## ✔ Signal orchestration

How multiple detectors feed a unified insight.

## ✔ Severity classification

How to make outputs actionable.

## ✔ Extensible utility design

Why this file is so easy to expand.

## ✔ Agent philosophy

Prediction is *input*, not *trigger*.
Detection is *trigger*, not prediction.

---

# 🔥 If You Want to Become Best-in-Class at Orchestrators

Here is your learning focus:

### ⭐ Learn to design **diagnostic panels** of signals

Gap detection = medical diagnostics, but for revenue.

### ⭐ Learn to tune thresholds based on business outcomes

Turning a 15% threshold into 18% can reduce noise dramatically.

### ⭐ Learn how to merge rule-based and ML-based signals

Most enterprise systems use BOTH.

### ⭐ Learn to generate crisp rationales

Orchestrators must *explain their reasoning* to be trusted.




In [ ]:
"""
Gap Detection Utilities for Revenue Gap Orchestrator

Detect revenue gaps, churn risks, and below-baseline performance.
"""

from typing import Dict, List, Any, Optional


def detect_declining_revenue_gap(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    threshold: float = -15.0
) -> Optional[Dict[str, Any]]:
    """
    Detect if customer has declining revenue gap.

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline data (from revenue analysis)
        threshold: Percentage decline threshold (e.g., -15.0 for 15% decline)

    Returns:
        Gap dictionary if detected, None otherwise
    """
    trend_percentage = revenue_baseline.get("trend_percentage", 0.0)
    revenue_trend = revenue_baseline.get("revenue_trend", "stable")

    # Check if declining and below threshold
    if revenue_trend == "declining" and trend_percentage <= threshold:
        recent_avg = revenue_baseline.get("recent_weeks_avg", 0.0)
        baseline_avg = revenue_baseline.get("baseline_weeks_avg", 0.0)
        gap_amount = recent_avg - baseline_avg

        return {
            "customer_id": customer_id,
            "gap_type": "declining_revenue",
            "current_revenue": recent_avg,
            "expected_revenue": baseline_avg,
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(trend_percentage, 2),
            "severity": _determine_severity(trend_percentage),
            "weeks_at_risk": 0,  # Will be calculated separately
            "rationale": f"Customer revenue declined {abs(trend_percentage):.1f}% from baseline"
        }

    return None


def detect_below_baseline_gap(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    current_revenue: float,
    threshold: float = -20.0
) -> Optional[Dict[str, Any]]:
    """
    Detect if customer is below baseline by threshold.

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline data
        current_revenue: Current period revenue (e.g., recent weeks average)
        threshold: Percentage below baseline threshold (e.g., -20.0 for 20% below)

    Returns:
        Gap dictionary if detected, None otherwise
    """
    baseline_avg = revenue_baseline.get("baseline_weeks_avg", 0.0)

    if baseline_avg <= 0:
        return None

    # Calculate percentage difference
    percentage_diff = ((current_revenue - baseline_avg) / baseline_avg) * 100

    # Check if below threshold
    if percentage_diff <= threshold:
        gap_amount = current_revenue - baseline_avg

        return {
            "customer_id": customer_id,
            "gap_type": "below_baseline",
            "current_revenue": round(current_revenue, 2),
            "expected_revenue": round(baseline_avg, 2),
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(percentage_diff, 2),
            "severity": _determine_severity(percentage_diff),
            "weeks_at_risk": 0,  # Will be calculated separately
            "rationale": f"Customer revenue {abs(percentage_diff):.1f}% below baseline"
        }

    return None


def detect_churn_risk(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    zero_weeks_threshold: int = 2
) -> Optional[Dict[str, Any]]:
    """
    Detect churn risk based on consecutive zero spend weeks.

    Args:
        customer_id: Customer ID
        sales_records: List of sales records (sorted by date, most recent last)
        zero_weeks_threshold: Number of consecutive zero spend weeks to flag

    Returns:
        Churn risk dictionary if detected, None otherwise
    """
    if not sales_records:
        return None

    # Sort by date to ensure correct order
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Count consecutive zero spend weeks from the end (most recent)
    consecutive_zeros = 0
    for record in reversed(sorted_records):
        weekly_spend = record.get('weekly_spend', 0.0)
        if weekly_spend == 0.0:
            consecutive_zeros += 1
        else:
            break

    # Check if threshold met
    if consecutive_zeros >= zero_weeks_threshold:
        # Calculate churn risk score (0-1)
        # More consecutive zeros = higher risk
        risk_score = min(1.0, consecutive_zeros / 4.0)  # Max risk at 4+ weeks

        risk_factors = []
        if consecutive_zeros >= zero_weeks_threshold:
            risk_factors.append("zero_spend_weeks")

        # Check if declining trend
        if len(sorted_records) >= 4:
            recent_avg = sum(r.get('weekly_spend', 0.0) for r in sorted_records[-4:]) / 4
            baseline_avg = sum(r.get('weekly_spend', 0.0) for r in sorted_records[:4]) / 4
            if recent_avg < baseline_avg * 0.7:  # 30% decline
                risk_factors.append("declining_trend")

        return {
            "customer_id": customer_id,
            "churn_risk_score": round(risk_score, 2),
            "risk_factors": risk_factors,
            "weeks_since_last_purchase": consecutive_zeros,
            "predicted_churn_probability": round(risk_score * 0.9, 2)  # Slightly conservative
        }

    return None


def detect_zero_spend_gap(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    current_period_weeks: int = 4
) -> Optional[Dict[str, Any]]:
    """
    Detect gap from zero spend in recent period.

    Args:
        customer_id: Customer ID
        sales_records: List of sales records
        current_period_weeks: Number of recent weeks to check

    Returns:
        Gap dictionary if zero spend detected, None otherwise
    """
    if not sales_records:
        return None

    # Sort by date
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Get recent weeks
    recent_records = sorted_records[-current_period_weeks:]

    # Check for zero spend
    zero_weeks = sum(1 for r in recent_records if r.get('weekly_spend', 0.0) == 0.0)

    if zero_weeks > 0:
        # Calculate expected revenue (baseline average)
        if len(sorted_records) >= 4:
            baseline_records = sorted_records[:4]
            expected_revenue = sum(r.get('weekly_spend', 0.0) for r in baseline_records) / len(baseline_records)
        else:
            expected_revenue = sum(r.get('weekly_spend', 0.0) for r in sorted_records) / len(sorted_records)

        current_revenue = 0.0  # Zero spend
        gap_amount = 0.0 - expected_revenue
        gap_percentage = -100.0 if expected_revenue > 0 else 0.0

        return {
            "customer_id": customer_id,
            "gap_type": "zero_spend",
            "current_revenue": current_revenue,
            "expected_revenue": round(expected_revenue, 2),
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(gap_percentage, 2),
            "severity": "high",
            "weeks_at_risk": zero_weeks,
            "rationale": f"Customer had {zero_weeks} zero spend week(s) in recent period"
        }

    return None


def _determine_severity(gap_percentage: float) -> str:
    """
    Determine gap severity based on percentage.

    Args:
        gap_percentage: Gap percentage (negative value)

    Returns:
        "high", "medium", or "low"
    """
    if gap_percentage <= -30.0:
        return "high"
    elif gap_percentage <= -15.0:
        return "medium"
    else:
        return "low"


def detect_all_gaps_for_customer(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    sales_records: List[Dict[str, Any]],
    gap_thresholds: Dict[str, Any]
) -> List[Dict[str, Any]]:
    """
    Detect all types of gaps for a single customer.

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline data
        sales_records: Sales records for customer
        gap_thresholds: Threshold configuration

    Returns:
        List of detected gaps
    """
    gaps = []

    # 1. Declining revenue gap
    declining_gap = detect_declining_revenue_gap(
        customer_id,
        revenue_baseline,
        threshold=gap_thresholds.get("declining_revenue_threshold", -15.0)
    )
    if declining_gap:
        gaps.append(declining_gap)

    # 2. Below baseline gap
    recent_avg = revenue_baseline.get("recent_weeks_avg", 0.0)
    below_baseline_gap = detect_below_baseline_gap(
        customer_id,
        revenue_baseline,
        current_revenue=recent_avg,
        threshold=gap_thresholds.get("below_baseline_threshold", -20.0)
    )
    if below_baseline_gap:
        gaps.append(below_baseline_gap)

    # 3. Zero spend gap
    zero_spend_gap = detect_zero_spend_gap(customer_id, sales_records)
    if zero_spend_gap:
        gaps.append(zero_spend_gap)

    return gaps


def detect_all_customers_gaps(
    customer_revenue_baseline: Dict[str, Dict[str, Any]],
    sales_lookup: Dict[str, List[Dict[str, Any]]],
    gap_thresholds: Dict[str, Any]
) -> List[Dict[str, Any]]:
    """
    Detect gaps for all customers.

    Args:
        customer_revenue_baseline: Revenue baseline data for all customers
        sales_lookup: Sales records lookup by customer_id
        gap_thresholds: Threshold configuration

    Returns:
        List of all detected gaps
    """
    all_gaps = []

    for customer_id, revenue_baseline in customer_revenue_baseline.items():
        sales_records = sales_lookup.get(customer_id, [])

        gaps = detect_all_gaps_for_customer(
            customer_id,
            revenue_baseline,
            sales_records,
            gap_thresholds
        )

        all_gaps.extend(gaps)

    return all_gaps


def detect_all_customers_churn_risk(
    sales_lookup: Dict[str, List[Dict[str, Any]]],
    zero_weeks_threshold: int = 2
) -> List[Dict[str, Any]]:
    """
    Detect churn risk for all customers.

    Args:
        sales_lookup: Sales records lookup by customer_id
        zero_weeks_threshold: Number of consecutive zero spend weeks to flag

    Returns:
        List of customers with churn risk
    """
    churn_risks = []

    for customer_id, sales_records in sales_lookup.items():
        churn_risk = detect_churn_risk(
            customer_id,
            sales_records,
            zero_weeks_threshold=zero_weeks_threshold
        )

        if churn_risk:
            churn_risks.append(churn_risk)

    return churn_risks



# Gap Detection Node

In [ ]:
def gap_detection_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Gap Detection Node: Detect revenue gaps and churn risks.

    Detects:
    - Declining revenue gaps
    - Below baseline gaps
    - Zero spend gaps
    - Churn risks
    """
    errors = state.get("errors", [])
    customer_revenue_baseline = state.get("customer_revenue_baseline")
    sales_lookup = state.get("sales_lookup")

    if not customer_revenue_baseline or not sales_lookup:
        return {
            "errors": errors + ["gap_detection_node: customer_revenue_baseline and sales_lookup required"]
        }

    # Get gap thresholds from state or config
    from config import PredictiveRevenueGapConfig
    config = PredictiveRevenueGapConfig()

    gap_thresholds = state.get("gap_thresholds") or config.gap_thresholds
    zero_weeks_threshold = gap_thresholds.get("churn_risk_zero_weeks", 2)

    try:
        # Detect all revenue gaps
        revenue_gaps = detect_all_customers_gaps(
            customer_revenue_baseline,
            sales_lookup,
            gap_thresholds
        )

        # Detect churn risks
        churn_risk_customers = detect_all_customers_churn_risk(
            sales_lookup,
            zero_weeks_threshold=zero_weeks_threshold
        )

        return {
            "revenue_gaps": revenue_gaps,
            "churn_risk_customers": churn_risk_customers,
            "errors": errors
        }

    except Exception as e:
        return {
            "errors": errors + [f"gap_detection_node: Unexpected error - {str(e)}"]
        }



# Tests for Gap Detection Utilities

In [ ]:
"""
Tests for Gap Detection Utilities

Testing Phase 4: Gap detection utilities before building the node
"""

import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(__file__).parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from agents.revenue_gap_orchestrator.utilities.gap_detection import (
    detect_declining_revenue_gap,
    detect_below_baseline_gap,
    detect_churn_risk,
    detect_zero_spend_gap,
    detect_all_gaps_for_customer,
    detect_all_customers_gaps,
    detect_all_customers_churn_risk,
    _determine_severity
)


def test_detect_declining_revenue_gap():
    """Test declining revenue gap detection"""
    revenue_baseline = {
        "revenue_trend": "declining",
        "trend_percentage": -25.0,
        "recent_weeks_avg": 30.0,
        "baseline_weeks_avg": 40.0
    }

    gap = detect_declining_revenue_gap("1", revenue_baseline, threshold=-15.0)

    assert gap is not None
    assert gap["customer_id"] == "1"
    assert gap["gap_type"] == "declining_revenue"
    assert gap["gap_percentage"] == -25.0
    assert gap["severity"] == "high"
    print("✅ Detect declining revenue gap test passed")


def test_detect_declining_revenue_gap_no_gap():
    """Test that stable revenue doesn't trigger gap"""
    revenue_baseline = {
        "revenue_trend": "stable",
        "trend_percentage": -5.0,
        "recent_weeks_avg": 38.0,
        "baseline_weeks_avg": 40.0
    }

    gap = detect_declining_revenue_gap("1", revenue_baseline, threshold=-15.0)

    assert gap is None
    print("✅ Detect declining revenue gap (no gap) test passed")


def test_detect_below_baseline_gap():
    """Test below baseline gap detection"""
    revenue_baseline = {
        "baseline_weeks_avg": 50.0
    }

    gap = detect_below_baseline_gap("1", revenue_baseline, current_revenue=30.0, threshold=-20.0)

    assert gap is not None
    assert gap["customer_id"] == "1"
    assert gap["gap_type"] == "below_baseline"
    assert gap["gap_percentage"] == -40.0  # (30-50)/50 * 100
    assert gap["severity"] == "high"
    print("✅ Detect below baseline gap test passed")


def test_detect_churn_risk():
    """Test churn risk detection"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 0.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 0.0},
    ]

    churn_risk = detect_churn_risk("1", sales_records, zero_weeks_threshold=2)

    assert churn_risk is not None
    assert churn_risk["customer_id"] == "1"
    assert churn_risk["weeks_since_last_purchase"] == 2
    assert churn_risk["churn_risk_score"] > 0
    assert "zero_spend_weeks" in churn_risk["risk_factors"]
    print("✅ Detect churn risk test passed")


def test_detect_churn_risk_no_risk():
    """Test that no zero weeks doesn't trigger churn risk"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 40.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 0.0},  # Only 1 zero week
    ]

    churn_risk = detect_churn_risk("1", sales_records, zero_weeks_threshold=2)

    assert churn_risk is None
    print("✅ Detect churn risk (no risk) test passed")


def test_detect_zero_spend_gap():
    """Test zero spend gap detection"""
    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 0.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 0.0},
    ]

    gap = detect_zero_spend_gap("1", sales_records)

    assert gap is not None
    assert gap["customer_id"] == "1"
    assert gap["gap_type"] == "zero_spend"
    assert gap["current_revenue"] == 0.0
    assert gap["severity"] == "high"
    print("✅ Detect zero spend gap test passed")


def test_determine_severity():
    """Test severity determination"""
    assert _determine_severity(-35.0) == "high"
    assert _determine_severity(-25.0) == "medium"
    assert _determine_severity(-10.0) == "low"
    print("✅ Determine severity test passed")


def test_detect_all_gaps_for_customer():
    """Test detecting all gaps for a customer"""
    revenue_baseline = {
        "revenue_trend": "declining",
        "trend_percentage": -25.0,
        "recent_weeks_avg": 30.0,
        "baseline_weeks_avg": 40.0
    }

    sales_records = [
        {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
        {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
        {"week_start_date": "2025-09-20", "weekly_spend": 30.0},
        {"week_start_date": "2025-09-27", "weekly_spend": 30.0},
    ]

    gap_thresholds = {
        "declining_revenue_threshold": -15.0,
        "below_baseline_threshold": -20.0,
        "churn_risk_zero_weeks": 2
    }

    gaps = detect_all_gaps_for_customer("1", revenue_baseline, sales_records, gap_thresholds)

    assert len(gaps) > 0
    assert any(gap["gap_type"] == "declining_revenue" for gap in gaps)
    print("✅ Detect all gaps for customer test passed")


def test_detect_all_customers_gaps():
    """Test detecting gaps for all customers"""
    customer_revenue_baseline = {
        "1": {
            "revenue_trend": "declining",
            "trend_percentage": -25.0,
            "recent_weeks_avg": 30.0,
            "baseline_weeks_avg": 40.0
        },
        "2": {
            "revenue_trend": "stable",
            "trend_percentage": -5.0,
            "recent_weeks_avg": 95.0,
            "baseline_weeks_avg": 100.0
        }
    }

    sales_lookup = {
        "1": [
            {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
            {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
            {"week_start_date": "2025-09-20", "weekly_spend": 30.0},
            {"week_start_date": "2025-09-27", "weekly_spend": 30.0},
        ],
        "2": [
            {"week_start_date": "2025-09-06", "weekly_spend": 100.0},
            {"week_start_date": "2025-09-13", "weekly_spend": 95.0},
            {"week_start_date": "2025-09-20", "weekly_spend": 100.0},
            {"week_start_date": "2025-09-27", "weekly_spend": 95.0},
        ]
    }

    gap_thresholds = {
        "declining_revenue_threshold": -15.0,
        "below_baseline_threshold": -20.0,
        "churn_risk_zero_weeks": 2
    }

    all_gaps = detect_all_customers_gaps(
        customer_revenue_baseline,
        sales_lookup,
        gap_thresholds
    )

    assert len(all_gaps) > 0
    assert any(gap["customer_id"] == "1" for gap in all_gaps)
    print("✅ Detect all customers gaps test passed")


def test_detect_all_customers_churn_risk():
    """Test detecting churn risk for all customers"""
    sales_lookup = {
        "1": [
            {"week_start_date": "2025-09-06", "weekly_spend": 50.0},
            {"week_start_date": "2025-09-13", "weekly_spend": 45.0},
            {"week_start_date": "2025-09-20", "weekly_spend": 0.0},
            {"week_start_date": "2025-09-27", "weekly_spend": 0.0},
        ],
        "2": [
            {"week_start_date": "2025-09-06", "weekly_spend": 100.0},
            {"week_start_date": "2025-09-13", "weekly_spend": 95.0},
            {"week_start_date": "2025-09-20", "weekly_spend": 100.0},
            {"week_start_date": "2025-09-27", "weekly_spend": 95.0},
        ]
    }

    churn_risks = detect_all_customers_churn_risk(sales_lookup, zero_weeks_threshold=2)

    assert len(churn_risks) > 0
    assert any(risk["customer_id"] == "1" for risk in churn_risks)
    assert not any(risk["customer_id"] == "2" for risk in churn_risks)
    print("✅ Detect all customers churn risk test passed")


if __name__ == "__main__":
    print("Testing Gap Detection Utilities...\n")

    test_detect_declining_revenue_gap()
    test_detect_declining_revenue_gap_no_gap()
    test_detect_below_baseline_gap()
    test_detect_churn_risk()
    test_detect_churn_risk_no_risk()
    test_detect_zero_spend_gap()
    test_determine_severity()
    test_detect_all_gaps_for_customer()
    test_detect_all_customers_gaps()
    test_detect_all_customers_churn_risk()

    print("\n✅ All gap detection utility tests passed!")



In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_034_Predictive_Revenue_Gap_Orchestrator % python3 tests/test_gap_detection_utilities.py
Testing Gap Detection Utilities...

✅ Detect declining revenue gap test passed
✅ Detect declining revenue gap (no gap) test passed
✅ Detect below baseline gap test passed
✅ Detect churn risk test passed
✅ Detect churn risk (no risk) test passed
✅ Detect zero spend gap test passed
✅ Determine severity test passed
✅ Detect all gaps for customer test passed
✅ Detect all customers gaps test passed
✅ Detect all customers churn risk test passed

✅ All gap detection utility tests passed!




# 🔥 **“Gaps are NOT predictions — They’re triggers.”**

This is the most important architectural idea in the entire system.

You are building something far more powerful than "analytics."

You are building an **agent**.

And agents do not just *predict*, they:

### **1️⃣ Analyze the environment**

### **2️⃣ Detect important signals**

### **3️⃣ Trigger actions based on those signals**

### **4️⃣ Prioritize those actions**

### **5️⃣ Execute or recommend actions**

This is the essence of an orchestrator.

Let’s break this down like a systems architect.

---

# 🧩 **1. Analysis → Detect → Trigger → Prioritize → Act**

Your revenue_gap_orchestrator is built around this exact pipeline:

### **➊ Analysis**

From the revenue utility:

* Baseline
* Trends
* Prediction (if available)
* Current spend
* Zero-spend windows
* Churn signals

Raw facts, NOT decisions.

### **➋ Detect**

Gap detectors turn raw facts into **patterns of concern**:

* “Revenue is dropping fast”
* “Customer is below baseline”
* “Customer isn't buying anything lately”
* “Stockout likely reduced purchasing”

This step converts **data → signals**.

### **➌ Trigger**

A trigger is produced when a rule says:

> "This pattern crosses a threshold and needs attention."

In your code:

* If the decline is >15% → trigger
* If the customer is 20% below baseline → trigger
* If 2+ weeks of zero spend → trigger
  All via:
  `detect_declining_revenue_gap()`
  `detect_below_baseline_gap()`
  `detect_zero_spend_gap()`
  `detect_churn_risk()`

Triggers create **events**, not predictions.

### **➍ Prioritize**

Not all gaps are equal.

Gaps feed into the next utility:

* Scoring
* Risk weighting
* Customer value weighting
* Gap type priority
* Potential recovery probability

The orchestrator decides:

> “Which 20 out of 200 customers should we act on first?”

### **➎ Act**

Finally, the orchestrator:

* Generates reports
* Updates CRM flags
* Sends tasks to a sales or marketing workflow
* (Eventually) uses an LLM to generate recommendations

---

# 🚨 **This Is Why Gaps ≠ Predictions**

**Predictions = tell me what *might* happen.**
**Gaps = tell me what I should *do* now.**

Predictions feed analysis.
Gap detection feeds orchestration.

This is the single most important architectural separation in agent systems.

---

# 🧠 **2. Why Agent Systems Need Triggers (Not Only Predictions)**

Let's illustrate with a question:

> If a model predicts a 12% chance of churn, should the agent take action?

Probably not.

But if:

* Customer has 3 zero-spend weeks
* Their revenue is down 28%
* They usually buy weekly
* They are a high-value customer
* Their store was out of stock for 2 weeks

Then the gap detector will produce multiple triggers:

* `zero_spend`
* `declining_revenue`
* `stockout_impact`

This becomes extremely actionable.

**Triggers are discrete.**
**Predictions are numeric.**

Orchestrators work best with discrete signals.

---

# 🏗️ **3. Gap Utilities = Enterprise Pattern Matching**

Every function in `gap_detection.py` is an example of enterprise detection engineering.

Let’s break the file down into the four categories of signals it detects:

## **A. Performance Failure Signals**

Like:

* Declining revenue
* Below baseline

These detect when performance is dropping enough to matter.

## **B. Behavioral Failure Signals**

Like:

* Zero spend gap
* Consecutive zero spend weeks
* Weeks-since-last-purchase

These detect changes in customer behavior.

## **C. Risk Signals**

Like:

* Churn risk scoring
* Several consecutive zero spend weeks

These detect if action is required NOW.

## **D. Explainability Signals**

Every gap returns a **rationale**:

* “…declined 22% from baseline”
* “…X zero-spend weeks”
* “…customer below expectation”

This is **LLM-ready**.

---

# 🔬 **4. Signal Fusion = What Makes It Powerful**

This is the key part:

Each detector is a **separate rule**, but the orchestrator:

### **Combines all of them into a unified insight.**

Example:

A customer might trigger:

* Zero spend gap
* Declining revenue gap
* Churn risk
* Below baseline

The orchestrator uses:

* scoring
* ranking
* customer value
* probability of recovery
  to create a **final ranked priority list**.

This is how enterprise systems work.

---

# 🚀 **5. Extensible Design: Add ML Without Breaking Anything**

Your gap utility is designed with *composability* in mind.

If you later add ML:

* churn model
* revenue forecast model
* lifetime value model
* anomaly detection model
* shopping mission classifier

They simply feed into **analysis**,
which then feeds into **gap detection**,
which then triggers the orchestrator.

You do **not** rewrite the detection pipeline.

You just create a new detector:

```python
def detect_ml_predicted_gap(customer_id, ml_prediction, threshold=0.7):
    if ml_prediction["churn_probability"] >= threshold:
        return {
            "customer_id": customer_id,
            "gap_type": "ai_predicted_churn",
            "severity": "high",
            "rationale": f"Model predicts {ml_prediction['churn_probability']:.2f} probability of churn"
        }
```

Drop it right into `detect_all_gaps_for_customer()`.

Everything else works as-is.

This is orchestration elegance.

---

# 🧠 **6. Agent Philosophy: Prediction ≠ Trigger**

### Prediction → informs understanding

### Trigger → informs action

Agents should never:

* act on predictions alone
* ignore rule-based thresholds
* make decisions without explainability

The beauty of your system is that **every detector returns a human-readable explanation**, which makes it:

* LLM-ready
* Business-friendly
* Transparent
* Auditable

This is EXACTLY how enterprise AI systems should work.

---

# 🎯 **Summary: What You Should Take Away**

By studying the gap utilities, you learn:

### ⭐ Enterprise orchestration design

### ⭐ How agents convert data → signals → triggers → action

### ⭐ How to separate predictions from decisions

### ⭐ How to build flexible, extensible detection systems

### ⭐ How to fuse multiple signals into a single insight

### ⭐ How to define rule-based business logic cleanly

### ⭐ How to build agent pipelines that scale

You’re learning **real agent engineering**, not just LLM prompt chaining.

